# Coursera Capstone Project

This notebook was part of the Coursera Data Science course and created by Alina Henne. Feel free to reach out in case you have any questions: **alina.henne@me.com** 

## Introduction

### 1.1 Background
A fictional aviation consulting company with headquarters in Stuttgart, Germany wants to expand to the US. They have been consulting European aviation businesses successfully for many years and now want to enhance American partnership and move closer to their clients overseas.

The company's culture is largely based on employee satisfaction, so in order to achieve the best possible working atmosphere for its employees and to synchronize the surroundings of the new company location with the surroundings of the headquarters (that have proved to be popular), a key criterium for choosing the location of their new offices in the US is that the surrounding venues should be as similar as possible.

### 1.2 Problem
An analysis of surrounding venues in Stuttgart compared to venues in different US cities with airports (for example Atlanta, Los Angeles, Chicago, New York and Seattle) is supposed to give insight into the question: *Based on similarity of surrounding venues, which US city is best for expanding the Stuttgart-based consulting firm?*

## 2. Data acquisition and cleaning

### 2.1 Data sources
For the analysis I will be using Foursquare location data for Stuttgart and US cities, including columns: 
- Venue type
- Venue longitude/latitude
- city longitude/latitude 

I will be comparing the venue types of the ten most common venues across the different cities and cluster them in order to find the cluster that includes Stuttgart (thus the choices of similar cities).
In order to find the list of US cities that shall be included in the analysis, I will also have a data set on US international airports and in what city they are. In order to not over-complicate this, I will only take the top 40 US airports (in terms of passenger numbers). The data set will be scraped from: https://www.world-airport-codes.com/us-top-40-airports.html.

To scrape the data, I have to first import all necessary libraries and then scrape the html data. This process is shown below.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

!pip install lxml

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [3]:
df = pd.read_html('https://www.world-airport-codes.com/us-top-40-airports.html')[0]
df.rename(columns={'Rank(2018)':'Rank', 'Airports (large hubs)':'Airport'}, inplace=True)
df.drop(columns=['2019', '2018', '2017', '2016'], inplace=True)
df.set_index('Rank', inplace=True)
df.head()

,Airport,IATACode,Major city served,State
Rank,,,,
1,Hartsfield–Jackson Atlanta International Airport,ATL,Atlanta,GA
2,Los Angeles International Airport,LAX,Los Angeles,CA
3,O'Hare International Airport,ORD,Chicago,IL
4,Dallas/Fort Worth International Airport,DFW,Dallas,TX
5,Denver International Airport,DEN,Denver,CO


Next step is to find all geographical coordinates of these cities. This, I will do with a simple loop.

In [4]:
addr = df['Major city served']+', '+df['State']
coord = pd.DataFrame(addr, columns = ['City']) 

lat = []
long = []
for var in addr:
    geolocator = Nominatim(user_agent=var+"_explorer")
    location = geolocator.geocode(var)
    latitude = location.latitude
    lat.append(latitude)
    longitude = location.longitude
    long.append(longitude)
    
coord["Latitude"] = lat
coord["Longitude"] = long

In [5]:
address = 'Stuttgart, Germany'

geolocator = Nominatim(user_agent="STR_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

new_row = {'City':address, 'Latitude':latitude, 'Longitude':longitude}
#append row to the dataframe
coord = coord.append(new_row, ignore_index=True)
coord

,City,Latitude,Longitude
0,"Atlanta, GA",33.749099,-84.390185
1,"Los Angeles, CA",34.053691,-118.242767
2,"Chicago, IL",41.875562,-87.624421
3,"Dallas, TX",32.776272,-96.796856
4,"Denver, CO",39.739236,-104.984862
5,"New York, NY",40.712728,-74.006015
6,"San Francisco, CA",46.844325,-71.274327
7,"Seattle, WA",47.603832,-122.330062
8,"Las Vegas, NV",36.167256,-115.148516
9,"Orlando, FL",28.542111,-81.379030


Now, I will create a function to pull venues nearby a specific location from Foursquare.

In [6]:
CLIENT_ID = '2BKVTMYZ3ALNVSDFEQUQVPMEJEFOUKQRBVCH4JJAZRPCB1RF' # my Foursquare ID
CLIENT_SECRET = 'GDKN1TOCOK12JHEKXO1LMFU5LY0ZWA5DQIPM3DKY4Z21YUDV' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

I can now use this function to pull nearby venues for all 41 cities.

In [7]:
airport_venues = getNearbyVenues(names=coord['City'],
                                   latitudes=coord['Latitude'],
                                   longitudes=coord['Longitude']                                  )

Atlanta, GA
Los Angeles, CA
Chicago, IL
Dallas, TX
Denver, CO
New York, NY
San Francisco, CA
Seattle, WA
Las Vegas, NV
Orlando, FL
New York, NJ
Charlotte, NC
Phoenix, AZ
Houston, TX
Miami, FL
Boston, MA
Minneapolis, MN
Fort Lauderdale, FL
Detroit, MI
Philadelphia, PA
New York, NY
Baltimore, MD
Salt Lake City, UT
San Diego, CA
Washington, D.C., VA
Washington, D.C., VA
Chicago, IL
Tampa, FL
Portland, OR
Honolulu, HI
Stuttgart, Germany


With one hot encoding and grouping as well as another function, I will now create a table that shows the top ten venues for each city:

In [8]:
# one hot encoding
airport_onehot = pd.get_dummies(airport_venues[['Venue Category']], prefix="", prefix_sep="")
#airport_onehot.drop(['City'], axis = 1,inplace=True)

# add city column back to dataframe
airport_onehot = pd.concat([airport_venues['City'],airport_onehot], axis = 1)
airport_onehot.head()

,City,ATM,Accessories Store,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Bookstore,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Empanada Restaurant,Escape Room,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Taverna,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train Station,Travel Lounge,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Atlanta, GA",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [9]:
airport_grouped = airport_onehot.groupby('City').mean().reset_index()
airport_grouped

,City,ATM,Accessories Store,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Basketball Court,College Bookstore,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Empanada Restaurant,Escape Room,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Taverna,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train Station,Travel Lounge,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Atlanta, GA",0.000000,0.000000,0.083333,0.00,0.00,0.083333,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.083333,0.000000,0.000000,0.00,0.000000,0.083333,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00000,0.000000,0.083333,0.000000,0.00,0.000000,0.00,0.000000

In [10]:
num_top_venues = 5

for city in airport_grouped['City']:
    print("----"+city+"----")
    temp = airport_grouped[airport_grouped['City'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Atlanta, GA----
                  venue  freq
0          Intersection  0.17
1           Event Space  0.08
2           Music Venue  0.08
3      Tapas Restaurant  0.08
4  Caribbean Restaurant  0.08


----Baltimore, MD----
                 venue  freq
0  American Restaurant  0.07
1          Coffee Shop  0.07
2          Music Venue  0.03
3               Lounge  0.03
4                Hotel  0.03


----Boston, MA----
                venue  freq
0  Italian Restaurant  0.15
1       Historic Site  0.07
2         Coffee Shop  0.06
3  Seafood Restaurant  0.06
4              Bakery  0.05


----Charlotte, NC----
                 venue  freq
0          Pizza Place  0.06
1           Steakhouse  0.05
2  American Restaurant  0.05
3                Hotel  0.05
4           Restaurant  0.04


----Chicago, IL----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.05
2           Hotel  0.04
3             Pub  0.03
4          Museum  0.03


----Dallas, TX----
         venue  freq
0      

In [11]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [12]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = airport_grouped['City']

for ind in np.arange(airport_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(airport_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Atlanta, GA",Intersection,Breakfast Spot,College Basketball Court,Food Court,Music Venue,Boat or Ferry,Caribbean Restaurant,Tapas Restaurant,Event Space,Art Gallery
1,"Baltimore, MD",American Restaurant,Coffee Shop,Lounge,Convenience Store,Café,Salad Place,Clothing Store,Music Venue,Hotel,Bar
2,"Boston, MA",Italian Restaurant,Historic Site,Seafood Restaurant,Coffee Shop,Bakery,Sandwich Place,Hotel,Pub,Park,Restaurant
3,"Charlotte, NC",Pizza Place,American Restaurant,Steakhouse,Hotel,Sandwich Place,Restaurant,Theater,Seafood Restaurant,Pub,French Restaurant
4,"Chicago, IL",Coffee Shop,Sandwich Place,Hotel,Pub,Boutique,Pizza Place,Bookstore,Museum,Concert Hall,Garden


___
Now, I can finally cluster and analyze.
___

In [13]:
# set number of clusters
kclusters = 15

airport_grouped_clustering = airport_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(airport_grouped_clustering)

# add clustering labels
city_venues_sorted.insert(0, 'Cluster', kmeans.labels_)

airport_merged = coord

# merge airport_grouped with coord to add latitude/longitude for each city
airport_merged = airport_merged.join(city_venues_sorted.set_index('City'), on='City')

airport_merged = airport_merged.fillna(0)
airport_merged['Cluster'] = airport_merged['Cluster'].astype('int')

airport_merged.head() # check the last columns!

,City,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Atlanta, GA",33.749099,-84.390185,7,Intersection,Breakfast Spot,College Basketball Court,Food Court,Music Venue,Boat or Ferry,Caribbean Restaurant,Tapas Restaurant,Event Space,Art Gallery
1,"Los Angeles, CA",34.053691,-118.242767,12,Coffee Shop,Museum,Breakfast Spot,Shopping Mall,Sushi Restaurant,Supermarket,Gift Shop,Speakeasy,Bookstore,Men's Store
2,"Chicago, IL",41.875562,-87.624421,6,Coffee Shop,Sandwich Place,Hotel,Pub,Boutique,Pizza Place,Bookstore,Museum,Concert Hall,Garden
3,"Dallas, TX",32.776272,-96.796856,4,Hotel,Coffee Shop,Café,Plaza,Bistro,IT Services,Furniture / Home Store,Mobile Phone Shop,Food Truck,Department Store
4,"Denver, CO",39.739236,-104.984862,6,Sandwich Place,Mexican Restaurant,Coffee Shop,Yoga Studio,Café,Pub,Pizza Place,Noodle House,Exhibit,Convenience Store


Above are the results. In order to interpret the results correctly, I will use a map to show the different clusters in different colors.

In [14]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(airport_merged['Latitude'], airport_merged['Longitude'], airport_merged['City'], airport_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

To get directly to the response (to the busiest airport in Stuttgart's cluster) I use basic pandas commands:

In [15]:
STR_cluster = airport_merged.loc[airport_merged['City'] == 'Stuttgart, Germany']['Cluster'].values
results = airport_merged[airport_merged.Cluster.isin(STR_cluster)]
results['City'].iloc[0]

'Chicago, IL'

In [ ]:
Feel free to look at the below table including all 

In [16]:
results

,City,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Chicago, IL",41.875562,-87.624421,6,Coffee Shop,Sandwich Place,Hotel,Pub,Boutique,Pizza Place,Bookstore,Museum,Concert Hall,Garden
4,"Denver, CO",39.739236,-104.984862,6,Sandwich Place,Mexican Restaurant,Coffee Shop,Yoga Studio,Café,Pub,Pizza Place,Noodle House,Exhibit,Convenience Store
5,"New York, NY",40.712728,-74.006015,6,Sandwich Place,Café,Coffee Shop,Hotel,American Restaurant,Cocktail Bar,Gym,Salad Place,Italian Restaurant,Spa
7,"Seattle, WA",47.603832,-122.330062,6,Coffee Shop,Hotel,Cocktail Bar,Café,Japanese Restaurant,Park,Italian Restaurant,Mexican Restaurant,Sushi Restaurant,Pizza Place
19,"Philadelphia, PA",39.952724,-75.163526,6,Hotel,Coffee Shop,Dessert Shop,Bakery,Donut Shop,Smoke Shop,Seafood Restaurant,Pub,Plaza,Clothing Store
20,"New York, NY",40.712728,-74.006015,6,Sandwich Place,Café,Coffee Shop,Hotel,American Restaurant,Cocktail Bar,Gym,Salad Place,Italian Restaurant,Spa
23,"San Diego, CA",32.717421,-117.162771,6,Hotel,Coffee Shop,Seafood Restaurant,Mexican Restaurant,Italian Restaurant,Sushi Restaurant,Clothing Store,Accessories Store,Sandwich Place,Bakery
26,"Chicago, IL",41.875562,-87.624421,6,Coffee Shop,Sandwich Place,Hotel,Pub,Boutique,Pizza Place,Bookstore,Museum,Concert Hall,Garden
30,"Stuttgart, Germany",48.778449,9.180013,6,German Restaurant,Plaza,Café,Sushi Restaurant,Clothing Store,Restaurant,History Museum,Bar,Gourmet Shop,Italian Restaurant
